In [ ]:
import matplotlib.pyplot as plt
import os
import re
import shutil
import string
import tensorflow as tf
import numpy as np
import requests
from bs4 import BeautifulSoup 

from keras import layers
from keras import losses

In [ ]:
import pandas as pd

filepath_dict = {'trainingData' : 'congressional_tweet_training_data.csv', 'testData' : 'congressional_tweet_training_data.csv'}

df_list = []
for source, filepath in filepath_dict.items():
    df = pd.read_csv(filepath, names=['favorite_count', 'full_text', 'hashtags', 'retweet_count', 'year', 'party_id'], low_memory=False, sep=',')
    df['source'] = source
    df_list.append(df)

df = pd.concat(df_list)
print(df.iloc[1])

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split


df_training = df[df['source'] == 'trainingData']

sentences = df_training['full_text'].values
y = df_training['party_id'].values

sentences_train, sentences_test, y_train, y_test = train_test_split(sentences, y, test_size=0.25, random_state=1000)

In [ ]:
from sklearn.linear_model import LogisticRegression

classifier = LogisticRegression(max_iter=1000)
classifier.fit(X_train, y_train)
score = classifier.score(X_test, y_test)

print("Accuracy: ", score)